In [13]:
import numpy as np
from sklearn.datasets import load_boston
from sklearn.model_selection import train_test_split
import torch


### Load dataset - Preprocessing
X, y = load_boston().data, load_boston().target
X = X.astype(np.float32)
y = y.reshape(-1, 1).astype(np.float32)
X_train, X_test, y_train, y_test = train_test_split(X, y,
                                                   test_size=0.2,
                                                   random_state=15)
# standardize data
#Xs = np.std(X_train, axis=0, keepdims=True)
#X_train /= Xs
#X_test /= Xs


### Build network
IN_SIZE = 13
HIDDEN_SIZE = 200
OUT_SIZE = 1
LR=1e-5

class Net(torch.nn.Module):

    def __init__(self):
        super(Net, self).__init__()
        self.l1 = torch.nn.Linear(IN_SIZE , HIDDEN_SIZE)
        self.l2 = torch.nn.Linear(HIDDEN_SIZE, OUT_SIZE)

    def forward(self, x):
        x = torch.relu(self.l1(x))
        y_logits = self.l2(x)
        return y_logits

net = Net()
criterion = torch.nn.MSELoss(reduction='mean')
opti = torch.optim.SGD(net.parameters(), lr=LR)

### Training
NEPOCHS = 500
BATCH_SIZE = 32

for epoch in range(NEPOCHS):

    p = np.random.permutation(len(X_train))
    X_train, y_train = X_train[p], y_train[p]
    for ki in range(0, len(X_train), BATCH_SIZE):
        X_batch = torch.from_numpy(X_train[ki:ki+BATCH_SIZE])
        y_batch = torch.from_numpy(y_train[ki:ki+BATCH_SIZE])
    
        net.zero_grad()
        y_logits = net(X_batch)
        loss = criterion(y_logits, y_batch)
        loss.backward()
        opti.step()
        
    if epoch % 10 == 0:
        X = torch.from_numpy(X_train)
        y = torch.from_numpy(y_train)
        y_logits = net(X)
        loss = criterion(y_logits, y)
        print('Epoch {}: Loss = {}'.format(epoch+1, loss.data))
    
    
### Evaluate
X = torch.from_numpy(X_test)
y = torch.from_numpy(y_test)
y_logits = net(X)
loss = criterion(y_logits, y)
print('Test Loss:', loss.data.numpy())

Epoch 1: Loss = 590.6029663085938
Epoch 11: Loss = 80.93412780761719
Epoch 21: Loss = 74.48310089111328
Epoch 31: Loss = 69.8826675415039
Epoch 41: Loss = 67.30660247802734
Epoch 51: Loss = 66.84329986572266
Epoch 61: Loss = 66.06513977050781
Epoch 71: Loss = 67.98539733886719
Epoch 81: Loss = 64.66675567626953
Epoch 91: Loss = 64.11405181884766
Epoch 101: Loss = 64.46937561035156
Epoch 111: Loss = 63.321136474609375
Epoch 121: Loss = 63.27306365966797
Epoch 131: Loss = 62.424015045166016
Epoch 141: Loss = 62.585784912109375
Epoch 151: Loss = 62.47132873535156
Epoch 161: Loss = 62.03402328491211
Epoch 171: Loss = 61.375343322753906
Epoch 181: Loss = 62.112205505371094
Epoch 191: Loss = 59.46728515625
Epoch 201: Loss = 59.07181930541992
Epoch 211: Loss = 58.49174880981445
Epoch 221: Loss = 60.270145416259766
Epoch 231: Loss = 57.60791015625
Epoch 241: Loss = 59.71610641479492
Epoch 251: Loss = 57.4201545715332
Epoch 261: Loss = 56.7076416015625
Epoch 271: Loss = 54.69457244873047
Epoch 